In [1]:
dataset = ''
#dataset1 = dataset
from pathlib import Path
output_root = Path("result")
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]

In [2]:
# Parameters
dataset = "log"


In [3]:
import pandas as pd
from lenskit import util
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lenskit.datasets import MovieLens
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import friedmanchisquare, wilcoxon
from itertools import combinations
from lenskit import util

In [4]:
output_root = Path("result")

In [5]:
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]

In [6]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 1000)

In [7]:
movies = pd.read_csv(r'C:\Users\srabantiguha\lk-demo-experiment\data\ml-20m\movies.csv')
movies = movies[['movieId','genres']]
movies = movies.join(movies.pop('genres').str.get_dummies('|'))
movies.rename({'movieId': 'item'}, axis=1, inplace=True)
movies

,item,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27273,131254,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27274,131256,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27275,131258,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27276,131260,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]
dirs

[WindowsPath('result/LOG-ALS'),
 WindowsPath('result/LOG-Bias'),
 WindowsPath('result/LOG-IALS'),
 WindowsPath('result/LOG-IIE'),
 WindowsPath('result/LOG-III'),
 WindowsPath('result/LOG-Pop'),
 WindowsPath('result/LOG-UUE')]

In [9]:
def calc_new_col(row):
    
    if row <= a :
        return 'underexposed'
    
    elif row>= b:
        return 'overexposed'
    
    else:
        return 'Neutral'

In [10]:
analy = pd.DataFrame(columns = ["dataset","shape","Dist_EEL", "quantile(0.25)","quantile(0.75)","accuracy"], index= range(len(dirs)))

for p in range(len(dirs)): 
    
    #print('Dataset:-',dirs[p])
    o = dirs[p]
    df = pd.read_csv(dirs[p], sep=',')
    #print(df.shape)

    df = df[['item','difference']]
#     print('Distribution of EEL')
    print(df['difference'].describe())

    a = df.difference.quantile(0.25) 
    b = df.difference.quantile(0.75) 
#     print("a and b")
#     print(a,b)

    df["category"] = df['difference'].apply(lambda x: calc_new_col(x))

    movies1 = movies.merge(df, on = 'item', how = 'inner')
    map1 = {'underexposed':-1, 'overexposed':1, 'Neutral':0}
    movies1['category'] = movies1['category'].map(map1)
    movies1 = movies1.drop(columns=['item', 'difference'])

    X = movies1.drop(columns=['category'])
    y = movies1['category']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)
    
    
    rf = RandomForestClassifier(random_state=42, n_jobs=-1)
    params = {
        'max_depth': [2,3,5,10,20],
        'min_samples_leaf': [5,10,20,50,100,200],
        'n_estimators': [10,25,30,50,100,200]
    }
    grid_search = GridSearchCV(estimator=rf,param_grid=params,cv = 4,n_jobs=-1, verbose=1, scoring="accuracy")
    grid_search.fit(X_train, y_train)
    y_pred_test = grid_search.predict(X_test)
    ac = (accuracy_score(y_test, y_pred_test))
    
    analy.iloc[p,0] = o
    analy.iloc[p,1] = (df.shape)
    analy.iloc[p,2] = 'Not showed'
    #analy.iloc[p,2] = df['difference'].describe()
    analy.iloc[p,3] = a
    analy.iloc[p,4] = b
    analy.iloc[p,5] = ac
    
analy1 = analy.T
analy1.columns = analy1.iloc[0]
analy1 =  analy1.drop(analy1.index[0])

count     1190.000000
mean      1203.928318
std       3592.599909
min      -2473.734012
25%         -1.819824
50%          6.920357
75%        471.079232
max      32679.731011
Name: difference, dtype: float64
Fitting 4 folds for each of 180 candidates, totalling 720 fits


count      476.000000
mean      6085.815530
std      12786.086713
min      -2973.952424
25%        -35.403442
50%         -0.766388
75%       2925.337498
max      97781.825015
Name: difference, dtype: float64
Fitting 4 folds for each of 180 candidates, totalling 720 fits


count     7346.000000
mean       344.305943
std        981.889508
min        -16.111225
25%         -1.233339
50%          7.395556
75%        139.605427
max      12709.402315
Name: difference, dtype: float64
Fitting 4 folds for each of 180 candidates, totalling 720 fits


count     9932.000000
mean        65.640579
std        422.538340
min      -2978.965015
25%         -1.309337
50%         16.436950
75%         98.579422
max      21508.523991
Name: difference, dtype: float64
Fitting 4 folds for each of 180 candidates, totalling 720 fits


count     3559.000000
mean       722.534055
std       3394.497702
min       -504.273197
25%        -52.126983
50%        -16.037396
75%         -3.124269
max      42480.329534
Name: difference, dtype: float64
Fitting 4 folds for each of 180 candidates, totalling 720 fits


count     1409.000000
mean      1856.811940
std       6721.438722
min       -536.033301
25%       -173.740648
50%       -106.551072
75%        -44.007897
max      73522.634087
Name: difference, dtype: float64
Fitting 4 folds for each of 180 candidates, totalling 720 fits


count      771.000000
mean        -9.542803
std       1510.221557
min      -3029.988083
25%        -71.845408
50%         -0.552750
75%          1.984576
max      30713.001175
Name: difference, dtype: float64
Fitting 4 folds for each of 180 candidates, totalling 720 fits


In [11]:
analy1

dataset,result\LOG-ALS,result\LOG-Bias,result\LOG-IALS,result\LOG-IIE,result\LOG-III,result\LOG-Pop,result\LOG-UUE
shape,"(1190, 3)","(476, 3)","(7346, 3)","(9932, 3)","(3559, 3)","(1409, 3)","(771, 3)"
Dist_EEL,Not showed,Not showed,Not showed,Not showed,Not showed,Not showed,Not showed
quantile(0.25),-1.819824,-35.403442,-1.233339,-1.309337,-52.126983,-173.740648,-71.845408
quantile(0.75),471.079232,2925.337498,139.605427,98.579422,-3.124269,-44.007897,1.984576
accuracy,0.479866,0.462185,0.501905,0.519936,0.48427,0.498584,0.528497
